# Introduction
We need to properly include lipid metabolism in the model. For this we will use the restrictive approach, where we need to ensure that we can produce the three lipid heads mentioned in beata's thesis. Then we need to generate a 'general'-acylcoa molecule that reflects the distribution of fatty acids found in te Tang et al. 2009 paper. We can then combine these two into the three 'average' glycerolipids and then include these into the biomass reaction.

For the branch chain fatty acids, we need to find some data that can specify in our strain what distribution of branches fatty acids we have. For now, in this notebook I will focus on first getting the phospholipid heads in place, then make sure the production of our 'general' acylcoa is correct and combine them together into our three lipids that can be added into biomass.

In this notebook I will tackle the unbranched fatty acids, and the anteiso-fatty acids that we need in the model. i will make a second notebook for the iso-fatty acids to prevent this one from getting way too long. 

In [2]:
import cameo
import pandas as pd
import cobra.io
from cobra import Reaction, Metabolite

In [3]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [3]:
model_e_coli = cameo.load_model('iML1515')

In [4]:
model_b_sub = cameo.load_model('iYO844')

## Glycerolipid heads
Here, from Beata's thesis we have data on the three main types of lipids present in our strain: phosphatidylglycerol, diphosphatidylglycerol and phosphatidyl ethanolamine. Here I will check that all three of these can be synthesized properly.

Note: Here I will make sure the metabolites and pathways are present. Probably, at a later time point when the generic acyl-tail has been defined we will need to revist the chemical formulas of the metabolites here to capture the different lipid types.

__Phosphatidyl glycerol__
can be made in a two step process from CDP-diacylglycerol which is present in out model as cdpdag_c.

In [4]:
#first add the phosphatidylglycerol metabolite
model.add_metabolites(Metabolite(id='pg_c', name = 'Phosphatidylglycerol', compartment = 'c', formula = 'C8H13O10PR2', charge = 0))

In [5]:
model.metabolites.pg_c.annotation['kegg.compound'] = 'C00344'
model.metabolites.pg_c.annotation['sbo'] = 'SBO:0000247'
model.metabolites.pg_c.annotation['chebi'] = 'CHEBI:17517'

In [6]:
#add phosphatidylglycerophosphate metabolite
model.add_metabolites(Metabolite(id='pgp_c', name = 'Phosphatidylglycerophosphate', compartment = 'c', formula = 'C8H14O13P2R2', charge = 0))

In [7]:
model.metabolites.pgp_c.annotation['kegg.compound'] = 'C03892'
model.metabolites.pgp_c.annotation['sbo'] = 'SBO:0000247'
model.metabolites.pgp_c.annotation['chebi'] = 'CHEBI:37393'

In [8]:
#add reaction from pgp to pg
model.add_reaction(Reaction(id='PGP', name = 'Phosphatidylglycerophosphate phosphohydrolase'))

In [9]:
model.reactions.PGP.annotation['kegg.reaction'] = 'R02029'
model.reactions.PGP.annotation['sbo'] = 'SBO:0000176'
model.reactions.PGP.annotation['ec-code'] = '3.1.3.27'

In [10]:
model.reactions.PGP.add_metabolites({
    model.metabolites.pgp_c:-1,
    model.metabolites.h2o_c:-1,
    model.metabolites.pg_c:1,
    model.metabolites.pi_c:1,
    model.metabolites.h_c: 2
})

In [11]:
#add reaction converting CDP-diacyl glycerol to pgp_c
model.add_reaction(Reaction(id='CDPPT', name = 'glycerophosphate phosphatidyltransferase;'))

In [12]:
model.reactions.CDPPT.annotation['kegg.reaction'] = 'R01801'
model.reactions.CDPPT.annotation['sbo'] = 'SBO:0000176'
model.reactions.CDPPT.annotation['ec-code'] = '2.7.8.5'

In [13]:
model.reactions.CDPPT.add_metabolites({
    model.metabolites.cdpdag_c:-1,
    model.metabolites.glyc3p_c:-1,
    model.metabolites.cmp_c:1,
    model.metabolites.pgp_c:1,
    model.metabolites.h_c:-3
})

In [14]:
model.metabolites.pa_EC_c.id = 'pa_c'

So now, conversion of cdpdag_c to pg_c is possible. But just to be sure, I will go further back and check cdpdag_c can be synthesized. It seems that this is linked to central metabolism and so I will leave it for now. 

In [16]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

__phosphatidyl ethanolamine__ should be made in two steps from CDP-dag too (via phosphatidyl-serine). Phosphatidyl serine can currently be made from CDPDAGPT reaction, from serine and cdpdag, which is correct. We just need to add the conversion from ps to phosphatidyl-ethanolamine. We also need to add the phosphoethanolamine metabolite . 

In [7]:
model.metabolites.ps_cho_c.id = 'ps_c'

In [8]:
#add phosphoethanolamine metabolite
model.add_metabolites(Metabolite(id='pe_c', name = 'Phosphatidylethanolamine', compartment = 'c', formula = 'C7H12NO8PR2', charge = 0))

In [9]:
model.metabolites.pe_c.annotation['kegg.compound'] = 'C00350'
model.metabolites.pe_c.annotation['sbo'] = 'SBO:0000247'
model.metabolites.pe_c.annotation['chebi'] = 'CHEBI:16038'

In [10]:
#add production of phosphoethanolamine
model.add_reaction(Reaction(id='PSD', name = 'phosphatidylserine decarboxylase'))

In [11]:
model.reactions.PSD.annotation['kegg.reaction'] = 'R02055'
model.reactions.PSD.annotation['sbo'] = 'SBO:0000176'
model.reactions.PSD.annotation['ec-code'] = '4.1.1.65'

In [12]:
model.reactions.PSD.add_metabolites({
    model.metabolites.ps_c:-1,
    model.metabolites.pe_c:1,
    model.metabolites.co2_c:1
})

In [13]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

__Diphosphatidylglycerol__ (AKA Cardiolipin) can be synthesized from phosphatidyl-glycerol. I will need to add the cardiolipin metabolite and conversion to it here.


In [14]:
#add cardiolipin metabolite
model.add_metabolites(Metabolite(id='clpn_c', name ='Cardiolipin', compartment = 'c', formula = 'C13H18O17P2R4', charge = 0))

In [15]:
model.metabolites.clpn_c.notes['NOTES'] = 'Cardiolipin; Diphosphatidylglycerol; 1,3-Bis(1,2-diacyl-sn-glycero-3-phospho)-sn-glycerol'

In [16]:
model.metabolites.clpn_c.annotation['kegg.compound'] = 'C05980'
model.metabolites.clpn_c.annotation['sbo'] = 'SBO:0000247'
model.metabolites.clpn_c.annotation['chebi'] = 'CHEBI:28494'

In [17]:
#add reaction
model.add_reaction(Reaction(id='CLPNS', name = 'Cardiolipin synthase'))

In [18]:
model.reactions.CLPNS.annotation['kegg.reaction'] = 'R07390'
model.reactions.CLPNS.annotation['sbo'] = 'SBO:0000176'
model.reactions.CLPNS.annotation['ec-code'] = '2.7.8.-'

In [19]:
model.reactions.CLPNS.add_metabolites({
    model.metabolites.pg_c:-2,
    model.metabolites.clpn_c:1,
    model.metabolites.glyc_c:1
})

In [20]:
reactions = [model.reactions.PGP, model.reactions.CDPPT, model.reactions.PSD, model.reactions.CLPNS]

In [21]:
model.groups.get_by_id('00564 - Glycerophospholipid metabolism').add_members(reactions)

In [22]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

## Acyl-CoA Tails
Now that the heads are in place, we need to make sure that each Acyl-CoA tail that has been detected in our data set can be made. From these different tail, we will then combine them with a specific data-defined stoichiometry into a general acyl-coa tail which will be used and combined with the different glycerolipid heads. 

So here I will go through the list of (branched) fatty acids in the supplementary data of Tang et al. 2009 and ensure their production.
All the tails we need to produce: 14:0 ISO, 14:0, 15:0 ISO,15:0 ANTEISO,15:0, 16:0 ISO, 16:0, 17:0 ISO,17:0 ANTEISO, 17:0, 18:4 w3c, 18:0           

First I will make sure that we can correctly produce Acyl-ACP and Malonyl-ACP which are needed to start the saturated unbranched fatty acid biosynthesis. 

Before I continue it is very important how we define these metabolites for mass balance. __The ACP-Chain WILL BE INDICATED AS C11H20N2O7PRS__. So all carbon chains should be added from the sulfur onwards. 

Note: After looking into our strains genome and data on similar species, we decided to include both the NADPH and NADH reductase in the last step of the elongation cycle. But to prevent the model from then just cylcing between the cofactors we need to restrict the reaction of this step to the required product. As no fatty acid metabolism is included in the model, this isn't such a problem. 

__Acyl-ACP__ Seems to be fine, except a duplicate reaction

In [5]:
#AMACT and KAS14 are duplicate reactions
model.remove_reactions(model.reactions.AMACT)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



__Malonyl-ACP__ both malcoa and malACP seem to be produced fine. 

### Even carbon saturated, unbranched Fatty Acids
Here, we first go to the production of butyryl-ACP through hexanoyl-ACP, and from there every 'cycle' add a C2 moeity (Malonyl-ACP) to the growing chain, all the way up to the final nC18:0 that we need. 

In [6]:
model.metabolites.butacp_c.id = 'butACP_c'

In [7]:
model.reactions.BUTACPAT.id = '3OAS60'

In [8]:
model.reactions.BUTACPOR.id = 'EAR40x'

In [9]:
model.reactions.EAR40x.bounds = (-1000,0)

In [10]:
# Add reaction with NADPH here too
model.add_reaction(Reaction(id='EAR40y', name = 'Enoyl-[acyl-carrier-protein] reductase (NADPH) (n-C4:0)'))

In [11]:
model.reactions.EAR40y.annotation = model_e_coli.reactions.EAR40y.annotation

In [12]:
model.reactions.EAR40y.add_metabolites({
    model.metabolites.but2eACP_c: -1,
    model.metabolites.h_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.butACP_c:1,
    model.metabolites.nadp_c:1
})

In [13]:
model.reactions.get_by_id('3HAD40_1').bounds = (0,1000)

In [14]:
#add conversion from 3hbutACP to actACP

In [15]:
model.add_reaction(Reaction(id='3OAR40_1', name ='3-oxoacyl-[acyl-carrier-protein] reductase (n-C4:0)'))

In [16]:
model.reactions.get_by_id('3OAR40_1').annotation['sbo'] = 'SBO:0000176'
model.reactions.get_by_id('3OAR40_1').annotation['kegg.reaction'] = 'R02767'
model.reactions.get_by_id('3OAR40_1').annotation['ec-code'] = '1.1.1.100'

In [17]:
model.reactions.get_by_id('3OAR40_1').add_metabolites({
    model.metabolites.get_by_id('3hbutACP_c'): 1,
    model.metabolites.actACP_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.h_c:-1
})

In [18]:
model.metabolites.get_by_id('3hbutACP_c').formula = 'C15H27N2O9PRS'

In [19]:
#missing KAS 15 reaction
model.add_reaction(Reaction(id='KAS15', name = 'Beta-ketoacyl-ACP synthase (2)'))

In [20]:
model.reactions.KAS15.annotation = model_e_coli.reactions.KAS15.annotation

In [21]:
model.reactions.KAS15.add_metabolites({
    model.metabolites.accoa_c:-1,
    model.metabolites.h_c:-1,
    model.metabolites.malACP_c:-1,
    model.metabolites.actACP_c:1,
    model.metabolites.co2_c:1, 
    model.metabolites.coa_c:1
})

In [22]:
#Synthesis up to butyryl-ACP
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

now move to C6:0 (hexanoyl-ACP)

In [23]:
model.metabolites.hexacp_c.id = 'hexACP_c'

In [24]:
model.metabolites.hexACP_c.name  = 'Hexanoyl-ACP (n-C6:0ACP)'

In [25]:
#wrong ID
model.reactions.HACPOR.id = 'EAR60x'

In [26]:
model.reactions.EAR60x.bounds = (-1000,0)

In [27]:
#finished up to hexanoyl-ACP
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

now onto octanoyl-ACP

In [28]:
model.reactions.HEXACPAT.id = '3OAS80'

In [29]:
model.reactions.get_by_id('3HAD80').bounds = (0,1000)

In [30]:
model.reactions.OCTACPOR.id = 'EAR80x'

In [31]:
model.reactions.EAR80x.bounds = (-1000,0)

In [32]:
model.metabolites.octacp_c.id = 'ocACP_c'

In [33]:
# add EAR80y reaction
model.add_reaction(Reaction(id='EAR80y', name = 'Enoyl-[acyl-carrier-protein] reductase (NADPH) (n-C8:0)'))

In [34]:
model.reactions.EAR80y.annotation = model_e_coli.reactions.EAR80y.annotation

In [35]:
model.reactions.EAR80y.add_metabolites({
    model.metabolites.h_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.toct2eACP_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.ocACP_c:1
})

In [36]:
#finished up to octanoyl-ACP
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Now on to C10:0.

In [37]:
model.reactions.OCTACPAT.id = '3OAS100'

In [38]:
model.reactions.get_by_id('3HAD100').bounds = (0,1000)

In [39]:
model.reactions.DECACPOR.id = 'EAR100x'

In [40]:
model.reactions.EAR100x.bounds = (-1000,0)

In [41]:
#add EAR100y
model.add_reaction(Reaction(id='EAR100y', name = 'Enoyl-[acyl-carrier-protein] reductase (NADPH) (n-C10:0)'))

In [42]:
model.reactions.EAR100y.annotation = model_e_coli.reactions.EAR100y.annotation

In [43]:
model.reactions.EAR100y.add_metabolites({
    model.metabolites.h_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.tdec2eACP_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.decanoylacp_c:1
})

In [44]:
model.metabolites.decanoylacp_c.id = 'dcaACP_c'

In [45]:
model.reactions.EAR100y.bounds = (0,1000)

In [46]:
#finished up to C10:0
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Onto C12:0

In [47]:
model.reactions.DECACPAT.id = '3OAS120'

In [48]:
model.reactions.get_by_id('3OAS120').bounds = (0,1000)

In [49]:
model.reactions.DDECACPOR.id = 'EAR120x'

In [50]:
model.reactions.EAR120x.bounds = (-1000,0)

In [51]:
#add EAR120y reaction
model.add_reaction(Reaction(id='EAR120y', name = 'Enoyl-[acyl-carrier-protein] reductase (NADPH) (n-C12:0)'))

In [52]:
model.reactions.EAR120y.annotation = model_e_coli.reactions.EAR120y.annotation

In [53]:
model.metabolites.dodecacp_c.id = 'ddcaACP_c'

In [54]:
model.reactions.EAR120y.add_metabolites({
    model.metabolites.h_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.tddec2eACP_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.ddcaACP_c :1
})

In [55]:
model.reactions.EAR120y.bounds = (0,1000)

In [56]:
#finished up to C12:0
#save& commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Now onto C14:0

In [57]:
model.reactions.MALACPAT.id = '3OAS140'

In [58]:
model.reactions.get_by_id('3HAD140').bounds = (0,1000)

In [59]:
model.reactions.MYRACPOR.id = 'EAR140x'

In [60]:
model.reactions.EAR140x.bounds = (-1000,0)

In [61]:
#add EAR140y reaction
model.add_reaction(Reaction(id='EAR140y', name = 'Enoyl-[acyl-carrier-protein] reductase (NADPH) (n-C14:0)'))

In [62]:
model.reactions.EAR140y.annotation = model_e_coli.reactions.EAR140y.annotation

In [63]:
model.reactions.EAR140y.bounds = (0,1000)

In [64]:
model.reactions.EAR140y.add_metabolites({
    model.metabolites.h_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.tmrs2eACP_c:-1,
    model.metabolites.myrsACP_c:1,
    model.metabolites.nadp_c:1
})

In [65]:
model.metabolites.myrsACP_c.name = 'Tetradecanoyl-[acp]'

In [66]:
#finished up to C14:0, 
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Now onto C16:0

In [67]:
model.reactions.MMACPAT.id = '3OAS160'

In [68]:
model.metabolites.get_by_id('3oxhdacp_c').id = '3opalmACP_c'

In [69]:
model.reactions.get_by_id('3HAD160').bounds = (0,1000)

In [70]:
model.reactions.OPALMACPOR.id = 'EAR160x'

In [71]:
model.reactions.EAR160x.bounds = (-1000,0)

In [72]:
model.metabolites.hdeacp_c.id = 'palmACP_c'

In [73]:
#add EAR160y
model.add_reaction(Reaction(id='EAR160y', name = 'Enoyl-[acyl-carrier-protein] reductase (NADPH) (n-C16:0)'))

In [74]:
model.reactions.EAR160y.annotation = model_e_coli.reactions.EAR160y.annotation

In [75]:
model.reactions.EAR160y.add_metabolites({
    model.metabolites.h_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.tpalm2eACP_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.palmACP_c:1
})

In [76]:
#finished until c16:0
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Now onto C18:0

In [77]:
model.reactions.HDEACPT.id = '3OAS180'

In [78]:
model.reactions.OXSTACPOR.id = '3OAR180'

In [79]:
model.reactions.get_by_id('3HAD180').bounds = (0,1000)

In [80]:
model.reactions.TOCTDACPOR.id = 'EAR180x'

In [81]:
model.reactions.EAR180x.bounds = (0,1000)

In [82]:
#add EAR180y
model.add_reaction(Reaction(id='EAR180y', name = 'Enoyl-[acyl-carrier-protein] reductase (NADPH) (n-C18:0)'))

In [83]:
model.reactions.EAR180y.annotation = model_e_coli.reactions.EAR180y.annotation

In [84]:
model.reactions.EAR180y.add_metabolites({
    model.metabolites.h_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.toctd2eACP_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.ocdcaACP_c:1
})

In [85]:
model.reactions.EAR180y.gene_reaction_rule = model.reactions.EAR180x.gene_reaction_rule

In [86]:
model.reactions.EAR160y.gene_reaction_rule = model.reactions.EAR160x.gene_reaction_rule
model.reactions.EAR140y.gene_reaction_rule = model.reactions.EAR140x.gene_reaction_rule
model.reactions.EAR120y.gene_reaction_rule = model.reactions.EAR120x.gene_reaction_rule
model.reactions.EAR100y.gene_reaction_rule = model.reactions.EAR100x.gene_reaction_rule
model.reactions.EAR80y.gene_reaction_rule = model.reactions.EAR80x.gene_reaction_rule
model.reactions.EAR60y.gene_reaction_rule = model.reactions.EAR60x.gene_reaction_rule
model.reactions.EAR40y.gene_reaction_rule = model.reactions.EAR40x.gene_reaction_rule

In [87]:
#add all new reactions to the fatty acid biosynthesis group
reactions = [model.reactions.EAR160y, model.reactions.EAR180y,model.reactions.EAR140y, model.reactions.EAR120y, model.reactions.EAR40y,model.reactions.EAR80y, model.reactions.get_by_id('3OAR40_1'), model.reactions.KAS15, model.reactions.EAR100y]

In [88]:
model.groups.get_by_id('00062 - Fatty acid elongation').add_members(reactions)

In [89]:
#finished up to C18:0
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Now I've finished making sure the pathway for the generation of the saturated, unbranched fatty acids is correct. Next I will tackle the unbranched, uneven carbon length fatty acids. 

### Unbranched, saturated odd-carbon fatty acids

These are made in the same way as the Even-chain saturated FAs, but the first molecule of the process is propionyl-CoA (C3) instead of acetyl-CoA (C2). So here I will go all the way from the C3 to the C17:0 pathway. 
So here, I will add all the steps in the pathway similarly to the even chain fatty acids.

Note: here I will not add these reactions to a group, as the groups we've used are defined by KEGG. These RCTs are not in kegg, and so i dont want to have confusion here and so will just not have them in the groups.

In [90]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [91]:
#first need to form propionyl-ACP from prorionyl-Coa, similar to the MCOATA reaction
#first add propionyl-ACP met
model.add_metabolites(Metabolite(id='propACP_c', name = 'Propionyl-ACP', charge =-1, compartment = 'c', formula = 'C14H25N2O8PRS'))

In [92]:
model.metabolites.propACP_c.annotation['sbo'] = 'SBO:0000247'

In [93]:
#add reaction
model.add_reaction(Reaction(id='PCOATA', name = 'Propanyl-CoA:[acyl-carrier-protein] S-malonyltransferase'))

In [94]:
model.reactions.PCOATA.annotation['sbo'] = 'SBO:0000176'

In [95]:
model.reactions.PCOATA.add_metabolites({
    model.metabolites.ppcoa_c:-1,
    model.metabolites.ACP_c:-1,
    model.metabolites.propACP_c:1,
    model.metabolites.coa_c:1,
    model.metabolites.h_c:1
})

In [96]:
model.reactions.PCOATA.bounds = (-1000,1000)

In [97]:
#add condenstation of prop-ACP with malonyl ACP to 3-oxopentanoyl-ACP
#add metabolite
model.add_metabolites(Metabolite(id='3opentACP_c', name = '3-oxopentanoyl-ACP', compartment = 'c', charge =-1, formula = 'C16H27O9N2PRS'))

In [98]:
model.metabolites.get_by_id('3opentACP_c').annotation['sbo'] = 'SBO:0000176'

In [99]:
#add reaction
model.add_reaction(Reaction(id='3OAS50', name = '3-oxopentanoyl-[acyl-carrier-protein] synthase (n-C5:0)'))

In [100]:
model.reactions.get_by_id('3OAS50').annotation['sbo'] = 'SBO:0000176'

In [101]:
#assume GPR is the same FAget_by_id('3OAS50')synthase: RTMO12345
model.reactions.get_by_id('3OAS50').gene_reaction_rule = 'RTMO12345'

In [102]:
model.reactions.get_by_id('3OAS50').add_metabolites({
    model.metabolites.malACP_c:-1,
    model.metabolites.propACP_c:-1,
    model.metabolites.ACP_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.get_by_id('3opentACP_c'):1,
    model.metabolites.h_c:-1
})

In [103]:
#next add reaction with NADPH
#first add 3-Hydroxypentanoyl-[acp] met
model.add_metabolites(Metabolite(id='3hpentACP_c', name = '3-Hydroxypentanoyl-[acp]', compartment = 'c', charge = -1, formula = 'C16H29O9N2PRS'))

In [104]:
model.metabolites.get_by_id('3hpentACP_c').annotation['sbo'] = 'SBO:0000247'

In [105]:
#add reaction
model.add_reaction(Reaction(id='3OAR50', name = ' 3-oxoacyl-[acyl-carrier-protein] reductase (n-C5:0)'))

In [106]:
model.reactions.get_by_id('3OAR50').annotation['sbo'] = 'SBO:0000176'

In [107]:
#assume GPR is the same FAget_by_id('3OAS50')synthase: RTMO12345
model.reactions.get_by_id('3OAR50').gene_reaction_rule = 'RTMO12345'

In [108]:
model.reactions.get_by_id('3OAR50').add_metabolites({
    model.metabolites.get_by_id('3opentACP_c'):-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3hpentACP_c'):1,
    model.metabolites.h_c:-1
})

In [109]:
#next add the tpent2eACP_c metabolite
model.add_metabolites(Metabolite(id='tpent2eACP_c', name = 'Trans-Pent-2-enoyl-[acyl-carrier protein]', compartment = 'c', charge = -1, formula = 'C16H27O8N2PRS'))

In [110]:
model.metabolites.get_by_id('tpent2eACP_c').annotation['sbo'] = 'SBO:0000247'

In [111]:
#add reaction
model.add_reaction(Reaction(id='3HAD50', name = '3-hydroxyacyl-[acyl-carrier-protein] dehydratase (n-C5:0)'))

In [112]:
model.reactions.get_by_id('3HAD50').annotation['sbo'] = 'SBO:0000176'

In [113]:
#assume GPR is the same FAget_by_id('3OAS50')synthase: RTMO12345
model.reactions.get_by_id('3HAD50').gene_reaction_rule = 'RTMO12345'

In [114]:
model.reactions.get_by_id('3HAD50').add_metabolites({
    model.metabolites.get_by_id('3hpentACP_c'):-1,
    model.metabolites.h2o_c:1,
    model.metabolites.tpent2eACP_c:1
})

In [115]:
#last step: convert to Heptanoyl-ACP which can start the next cycle again
#add both with NADH and NADPH.
#first add the heptanoyl-ACP metabolite
model.add_metabolites(Metabolite(id='pentACP_c', name = 'pentanoyl-ACP(n-C5:0ACP)', compartment = 'c', charge = -1, formula = 'C16H29O8N2PRS' ))

In [116]:
model.metabolites.pentACP_c.annotation['sbo'] = 'SBO:0000247'

In [117]:
#add reaction with NADH: EAR50x
model.add_reaction(Reaction(id='EAR50x', name = 'pentanoyl-[acp]:NAD+ trans-2-oxidoreductase'))

In [118]:
model.reactions.EAR50x.annotation['sbo'] = 'SBO:0000176'

In [119]:
#assume GPR is the same FAget_by_id('3OAS50')synthase: RTMO12345
model.reactions.EAR50x.gene_reaction_rule = 'RTMO12345'

In [120]:
model.reactions.EAR50x.add_metabolites({
    model.metabolites.tpent2eACP_c:-1,
    model.metabolites.nadh_c:-1,
    model.metabolites.nad_c:1,
    model.metabolites.pentACP_c:1,
    model.metabolites.h_c:-1
})

In [121]:
#add reaction with NADH: EAR50Y
model.add_reaction(Reaction(id='EAR50y', name = 'pentanoyl-[acp]:NADp+ trans-2-oxidoreductase'))

In [122]:
model.reactions.EAR50y.annotation['sbo'] = 'SBO:0000176'

In [123]:
#assume GPR is the same FAget_by_id('3OAS50')synthase: RTMO12345
model.reactions.EAR50y.gene_reaction_rule = 'RTMO12345'

In [124]:
model.reactions.EAR50y.add_metabolites({
    model.metabolites.tpent2eACP_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.pentACP_c:1,
    model.metabolites.h_c:-1
})

In [125]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Now I finished the cycle until nC5:0. I will save and commit, and add this cycle multiple times until we get all the way to C17:0.

In [126]:
#add condenstation of pent-ACP with malonyl ACP to 3-oxoheptanoyl-ACP
#add metabolite
model.add_metabolites(Metabolite(id='3oheptACP_c', name = '3-oxoheptanoyl-ACP', compartment = 'c', charge =-1, formula = 'C18H31O9N2PRS'))

In [127]:
model.metabolites.get_by_id('3oheptACP_c').annotation['sbo'] = 'SBO:0000176'

In [128]:
#add reaction
model.add_reaction(Reaction(id='3OAS70', name = '3-oxoheptanoyl-[acyl-carrier-protein] synthase (n-C7:0)'))

In [129]:
model.reactions.get_by_id('3OAS70').annotation['sbo'] = 'SBO:0000176'

In [130]:
#assume GPR is the same FAget_by_id('3OAS50')synthase: RTMO12345
model.reactions.get_by_id('3OAS70').gene_reaction_rule = 'RTMO12345'

In [131]:
model.reactions.get_by_id('3OAS70').add_metabolites({
    model.metabolites.malACP_c:-1,
    model.metabolites.pentACP_c:-1,
    model.metabolites.ACP_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.get_by_id('3oheptACP_c'):1,
    model.metabolites.h_c:-1
})

In [132]:
#next add reaction with NADPH
#first add 3-Hydroxypentanoyl-[acp] met
model.add_metabolites(Metabolite(id='3hheptACP_c', name = '3-Hydroxyheptanoyl-[acp]', compartment = 'c', charge = -1, formula = 'C18H33O9N2PRS'))

In [133]:
model.metabolites.get_by_id('3hheptACP_c').annotation['sbo'] = 'SBO:0000247'

In [134]:
#add reaction
model.add_reaction(Reaction(id='3OAR70', name = ' 3-oxoacyl-[acyl-carrier-protein] reductase (n-C7:0)'))

In [135]:
model.reactions.get_by_id('3OAR70').annotation['sbo'] = 'SBO:0000176'

In [136]:
#assume GPR is the same FAget_by_id('3OAS70')synthase: RTMO12345
model.reactions.get_by_id('3OAR70').gene_reaction_rule = 'RTMO12345'

In [137]:
model.reactions.get_by_id('3OAR70').add_metabolites({
    model.metabolites.get_by_id('3oheptACP_c'):-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3hheptACP_c'):1,
    model.metabolites.h_c:-1
})

In [138]:
#next add the tpent2eACP_c metabolite
model.add_metabolites(Metabolite(id='thept2eACP_c', name = 'Trans-Hept-2-enoyl-[acyl-carrier protein]', compartment = 'c', charge = -1, formula = 'C18H31O8N2PRS'))

In [139]:
model.metabolites.get_by_id('thept2eACP_c').annotation['sbo'] = 'SBO:0000247'

In [140]:
#add reaction
model.add_reaction(Reaction(id='3HAD70', name = '3-hydroxyacyl-[acyl-carrier-protein] dehydratase (n-C7:0)'))

In [141]:
model.reactions.get_by_id('3HAD70').annotation['sbo'] = 'SBO:0000176'

In [142]:
#assume GPR is the same FAget_by_id('3OAS50')synthase: RTMO12345
model.reactions.get_by_id('3HAD70').gene_reaction_rule = 'RTMO12345'

In [143]:
model.reactions.get_by_id('3HAD70').add_metabolites({
    model.metabolites.get_by_id('3hheptACP_c'):-1,
    model.metabolites.h2o_c:1,
    model.metabolites.thept2eACP_c:1
})

In [144]:
#last step: convert to Heptanoyl-ACP which can start the next cycle again
#add both with NADH and NADPH.
#first add the heptanoyl-ACP metabolite
model.add_metabolites(Metabolite(id='heptACP_c', name = 'heptanoyl-ACP(n-C7:0ACP)', compartment = 'c', charge = -1, formula = 'C18H33O8N2PRS' ))

In [145]:
model.metabolites.heptACP_c.annotation['sbo'] = 'SBO:0000247'

In [146]:
#add reaction with NADH
model.add_reaction(Reaction(id='EAR70x', name = 'heptanoyl-[acp]:NAD+ trans-2-oxidoreductase'))

In [147]:
model.reactions.EAR70x.annotation['sbo'] = 'SBO:0000176'

In [148]:
#assume GPR is the same FAget_by_id('3OAS50')synthase: RTMO12345
model.reactions.EAR70x.gene_reaction_rule = 'RTMO12345'

In [149]:
model.reactions.EAR70x.add_metabolites({
    model.metabolites.thept2eACP_c:-1,
    model.metabolites.nadh_c:-1,
    model.metabolites.nad_c:1,
    model.metabolites.heptACP_c:1,
    model.metabolites.h_c:-1
})

In [150]:
#add reaction with NADH
model.add_reaction(Reaction(id='EAR70y', name = 'heptanoyl-[acp]:NADP+ trans-2-oxidoreductase'))

In [151]:
model.reactions.EAR70y.annotation['sbo'] = 'SBO:0000176'

In [152]:
#assume GPR is the same FAget_by_id('3OAS70')synthase: RTMO12345
model.reactions.EAR70y.gene_reaction_rule = 'RTMO12345'

In [153]:
model.reactions.EAR70y.add_metabolites({
    model.metabolites.thept2eACP_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.heptACP_c:1,
    model.metabolites.h_c:-1
})

In [154]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Finished up to n-C7:0. Now onto n-C9:0.

In [ ]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [6]:
#add condenstation of hept-ACP with malonyl ACP to 3-oxononanoyl-ACP
#add metabolite
model.add_metabolites(Metabolite(id='3ononACP_c', name = '3-oxononanoyl-ACP', compartment = 'c', charge =-1, formula = 'C20H35O9N2PRS'))

In [7]:
model.metabolites.get_by_id('3ononACP_c').annotation['sbo'] = 'SBO:0000176'

In [9]:
#add reaction
model.add_reaction(Reaction(id='3OAS90', name = '3-oxononanoyl-[acyl-carrier-protein] synthase (n-C9:0)'))

In [10]:
model.reactions.get_by_id('3OAS90').annotation['sbo'] = 'SBO:0000176'

In [11]:
#assume GPR is the same FAget_by_id('3OAS50')synthase: RTMO12345
model.reactions.get_by_id('3OAS90').gene_reaction_rule = 'RTMO12345'

In [12]:
model.reactions.get_by_id('3OAS90').add_metabolites({
    model.metabolites.malACP_c:-1,
    model.metabolites.heptACP_c:-1,
    model.metabolites.ACP_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.get_by_id('3ononACP_c'):1,
    model.metabolites.h_c:-1
})

In [13]:
#next add reaction with NADPH
#first add 3-Hydroxynonanoyl-[acp] met
model.add_metabolites(Metabolite(id='3hnonACP_c', name = '3-Hydroxynonanoyl-[acp]', compartment = 'c', charge = -1, formula = 'C20H37O9N2PRS'))

In [14]:
model.metabolites.get_by_id('3hnonACP_c').annotation['sbo'] = 'SBO:0000247'

In [15]:
#add reaction
model.add_reaction(Reaction(id='3OAR90', name = ' 3-oxoacyl-[acyl-carrier-protein] reductase (n-C9:0)'))

In [16]:
model.reactions.get_by_id('3OAR90').annotation['sbo'] = 'SBO:0000176'

In [17]:
#assume GPR is the same FA synthase, as done in the whole model: RTMO12345
model.reactions.get_by_id('3OAR90').gene_reaction_rule = 'RTMO12345'

In [18]:
model.reactions.get_by_id('3OAR90').add_metabolites({
    model.metabolites.get_by_id('3ononACP_c'):-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3hnonACP_c'):1,
    model.metabolites.h_c:-1
})

In [19]:
#next add the tpent2eACP_c metabolite
model.add_metabolites(Metabolite(id='tnon2eACP_c', name = 'Trans-Non-2-enoyl-[acyl-carrier protein]', compartment = 'c', charge = -1, formula = 'C20H35O8N2PRS'))

In [20]:
model.metabolites.get_by_id('tnon2eACP_c').annotation['sbo'] = 'SBO:0000247'

In [21]:
#add reaction
model.add_reaction(Reaction(id='3HAD90', name = '3-hydroxyacyl-[acyl-carrier-protein] dehydratase (n-C9:0)'))

In [22]:
model.reactions.get_by_id('3HAD90').annotation['sbo'] = 'SBO:0000176'

In [23]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3HAD90').gene_reaction_rule = 'RTMO12345'

In [24]:
model.reactions.get_by_id('3HAD90').add_metabolites({
    model.metabolites.get_by_id('3hnonACP_c'):-1,
    model.metabolites.h2o_c:1,
    model.metabolites.tnon2eACP_c:1
})

In [25]:
#add both with NADH and NADPH.
#first add the nonanoyl-ACP metabolite
model.add_metabolites(Metabolite(id='nonACP_c', name = 'nonanoyl-ACP(n-C9:0ACP)', compartment = 'c', charge = -1, formula = 'C20H37O8N2PRS' ))

In [26]:
model.metabolites.nonACP_c.annotation['sbo'] = 'SBO:0000247'

In [27]:
#add reaction with NADH
model.add_reaction(Reaction(id='EAR90x', name = 'nonanoyl-[acp]:NAD+ trans-2-oxidoreductase'))

In [28]:
model.reactions.EAR90x.annotation['sbo'] = 'SBO:0000176'

In [29]:
#assume GPR is the same FAget_by_id('3OAS50')synthase: RTMO12345
model.reactions.EAR90x.gene_reaction_rule = 'RTMO12345'

In [30]:
model.reactions.EAR90x.add_metabolites({
    model.metabolites.tnon2eACP_c:-1,
    model.metabolites.nadh_c:-1,
    model.metabolites.nad_c:1,
    model.metabolites.nonACP_c:1,
    model.metabolites.h_c:-1
})

In [31]:
#add reaction with NADH
model.add_reaction(Reaction(id='EAR90y', name = 'nonanoyl-[acp]:NADP+ trans-2-oxidoreductase'))

In [32]:
model.reactions.EAR90y.annotation['sbo'] = 'SBO:0000176'

In [33]:
#assume GPR is the same FAget_by_id('3OAS70')synthase: RTMO12345
model.reactions.EAR90y.gene_reaction_rule = 'RTMO12345'

In [34]:
model.reactions.EAR90y.add_metabolites({
    model.metabolites.tnon2eACP_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.nonACP_c:1,
    model.metabolites.h_c:-1
})

In [35]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Finished up to nC9:0. Now onto nC11:0.

In [36]:
#add condenstation of non-ACP with malonyl ACP to 3-oxoundecanoyl-ACP
#add metabolite
model.add_metabolites(Metabolite(id='3oudecACP_c', name = '3-oxoundecanoyl-ACP', compartment = 'c', charge =-1, formula = 'C22H39O9N2PRS'))

In [37]:
model.metabolites.get_by_id('3oudecACP_c').annotation['sbo'] = 'SBO:0000176'

In [38]:
#add reaction
model.add_reaction(Reaction(id='3OAS110', name = '3-oxoundecanoyl-[acyl-carrier-protein] synthase (n-C11:0)'))

In [39]:
model.reactions.get_by_id('3OAS110').annotation['sbo'] = 'SBO:0000176'

In [40]:
#assume GPR is the same FAget_by_id('3OAS50')synthase: RTMO12345
model.reactions.get_by_id('3OAS110').gene_reaction_rule = 'RTMO12345'

In [41]:
model.reactions.get_by_id('3OAS110').add_metabolites({
    model.metabolites.malACP_c:-1,
    model.metabolites.nonACP_c:-1,
    model.metabolites.ACP_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.get_by_id('3oudecACP_c'):1,
    model.metabolites.h_c:-1
})

In [42]:
#next add reaction with NADPH
#first add 3-Hydroxyundecanoyl-[acp] met
model.add_metabolites(Metabolite(id='3hudecACP_c', name = '3-Hydroxyundecanoyl-[acp]', compartment = 'c', charge = -1, formula = 'C22H41O9N2PRS'))

In [43]:
model.metabolites.get_by_id('3hudecACP_c').annotation['sbo'] = 'SBO:0000247'

In [44]:
#add reaction
model.add_reaction(Reaction(id='3OAR110', name = ' 3-oxoacyl-[acyl-carrier-protein] reductase (n-C11:0)'))

In [45]:
model.reactions.get_by_id('3OAR110').annotation['sbo'] = 'SBO:0000176'

In [46]:
#assume GPR is the same FA synthase, as done in the whole model: RTMO12345
model.reactions.get_by_id('3OAR110').gene_reaction_rule = 'RTMO12345'

In [47]:
model.reactions.get_by_id('3OAR110').add_metabolites({
    model.metabolites.get_by_id('3oudecACP_c'):-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3hudecACP_c'):1,
    model.metabolites.h_c:-1
})

In [48]:
#next add the tpent2eACP_c metabolite
model.add_metabolites(Metabolite(id='tudec2eACP_c', name = 'Trans-Undec-2-enoyl-[acyl-carrier protein]', compartment = 'c', charge = -1, formula = 'C22H39O8N2PRS'))

In [49]:
model.metabolites.get_by_id('tudec2eACP_c').annotation['sbo'] = 'SBO:0000247'

In [50]:
#add reaction
model.add_reaction(Reaction(id='3HAD110', name = '3-hydroxyacyl-[acyl-carrier-protein] dehydratase (n-C11:0)'))

In [51]:
model.reactions.get_by_id('3HAD110').annotation['sbo'] = 'SBO:0000176'

In [52]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3HAD110').gene_reaction_rule = 'RTMO12345'

In [53]:
model.reactions.get_by_id('3HAD110').add_metabolites({
    model.metabolites.get_by_id('3hudecACP_c'):-1,
    model.metabolites.h2o_c:1,
    model.metabolites.tudec2eACP_c:1
})

In [54]:
#add both with NADH and NADPH.
#first add the undecanoyl-ACP metabolite
model.add_metabolites(Metabolite(id='udecACP_c', name = 'Undecanoyl-ACP(n-C11:0ACP)', compartment = 'c', charge = -1, formula = 'C22H41O8N2PRS' ))

In [55]:
model.metabolites.udecACP_c.annotation['sbo'] = 'SBO:0000247'

In [56]:
#add reaction with NADH
model.add_reaction(Reaction(id='EAR110x', name = 'Undecanoyl-[acp]:NAD+ trans-2-oxidoreductase'))

In [57]:
model.reactions.EAR110x.annotation['sbo'] = 'SBO:0000176'

In [58]:
#assume GPR is the same FAget_by_id('3OAS50')synthase: RTMO12345
model.reactions.EAR110x.gene_reaction_rule = 'RTMO12345'

In [59]:
model.reactions.EAR110x.add_metabolites({
    model.metabolites.tudec2eACP_c:-1,
    model.metabolites.nadh_c:-1,
    model.metabolites.nad_c:1,
    model.metabolites.udecACP_c:1,
    model.metabolites.h_c:-1
})

In [60]:
#add reaction with NADH
model.add_reaction(Reaction(id='EAR110y', name = 'Undecanoyl-[acp]:NADP+ trans-2-oxidoreductase'))

In [61]:
model.reactions.EAR110y.annotation['sbo'] = 'SBO:0000176'

In [62]:
#assume GPR is the same FAget_by_id('3OAS70')synthase: RTMO12345
model.reactions.EAR110y.gene_reaction_rule = 'RTMO12345'

In [63]:
model.reactions.EAR110y.add_metabolites({
    model.metabolites.tudec2eACP_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.udecACP_c:1,
    model.metabolites.h_c:-1
})

In [64]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Now finished up to C11:0, move onto C13:0.

In [65]:
#add condenstation of non-ACP with malonyl ACP to 3-oxoundecanoyl-ACP
#add metabolite
model.add_metabolites(Metabolite(id='3otdecACP_c', name = '3-oxotridecanoyl-ACP', compartment = 'c', charge =-1, formula = 'C24H43O9N2PRS'))

In [66]:
model.metabolites.get_by_id('3otdecACP_c').annotation['sbo'] = 'SBO:0000176'

In [67]:
#add reaction
model.add_reaction(Reaction(id='3OAS130', name = '3-oxotridecanoyl-[acyl-carrier-protein] synthase (n-C13:0)'))

In [68]:
model.reactions.get_by_id('3OAS130').annotation['sbo'] = 'SBO:0000176'

In [69]:
#assume GPR is the same FAget_by_id('3OAS50')synthase: RTMO12345
model.reactions.get_by_id('3OAS130').gene_reaction_rule = 'RTMO12345'

In [70]:
model.reactions.get_by_id('3OAS130').add_metabolites({
    model.metabolites.malACP_c:-1,
    model.metabolites.udecACP_c:-1,
    model.metabolites.ACP_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.get_by_id('3otdecACP_c'):1,
    model.metabolites.h_c:-1
})

In [71]:
#next add reaction with NADPH
#first add 3-Hydroxytridecanoyl-[acp] met
model.add_metabolites(Metabolite(id='3htdecACP_c', name = '3-Hydroxytridecanoyl-[acp]', compartment = 'c', charge = -1, formula = 'C24H45O9N2PRS'))

In [72]:
model.metabolites.get_by_id('3htdecACP_c').annotation['sbo'] = 'SBO:0000247'

In [73]:
#add reaction
model.add_reaction(Reaction(id='3OAR130', name = ' 3-oxoacyl-[acyl-carrier-protein] reductase (n-C13:0)'))

In [74]:
model.reactions.get_by_id('3OAR130').annotation['sbo'] = 'SBO:0000176'

In [75]:
#assume GPR is the same FA synthase, as done in the whole model: RTMO12345
model.reactions.get_by_id('3OAR130').gene_reaction_rule = 'RTMO12345'

In [76]:
model.reactions.get_by_id('3OAR130').add_metabolites({
    model.metabolites.get_by_id('3otdecACP_c'):-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3htdecACP_c'):1,
    model.metabolites.h_c:-1
})

In [77]:
#next add the tpent2eACP_c metabolite
model.add_metabolites(Metabolite(id='ttdec2eACP_c', name = 'Trans-Tridec-2-enoyl-[acyl-carrier protein]', compartment = 'c', charge = -1, formula = 'C24H43O8N2PRS'))

In [78]:
model.metabolites.get_by_id('ttdec2eACP_c').annotation['sbo'] = 'SBO:0000247'

In [79]:
#add reaction
model.add_reaction(Reaction(id='3HAD130', name = '3-hydroxyacyl-[acyl-carrier-protein] dehydratase (n-C13:0)'))

In [80]:
model.reactions.get_by_id('3HAD130').annotation['sbo'] = 'SBO:0000176'

In [81]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3HAD130').gene_reaction_rule = 'RTMO12345'

In [82]:
model.reactions.get_by_id('3HAD130').add_metabolites({
    model.metabolites.get_by_id('3htdecACP_c'):-1,
    model.metabolites.h2o_c:1,
    model.metabolites.ttdec2eACP_c:1
})

In [83]:
#add the last reaction both with NADH and NADPH.
#first add the tridecanoyl-ACP metabolite
model.add_metabolites(Metabolite(id='tdecACP_c', name = 'Tridecanoyl-ACP(n-C13:0ACP)', compartment = 'c', charge = -1, formula = 'C24H45O8N2PRS' ))

In [84]:
model.metabolites.tdecACP_c.annotation['sbo'] = 'SBO:0000247'

In [85]:
#add reaction with NADH
model.add_reaction(Reaction(id='EAR130x', name = 'Tridecanoyl-[acp]:NAD+ trans-2-oxidoreductase'))

In [86]:
model.reactions.EAR130x.annotation['sbo'] = 'SBO:0000176'

In [87]:
#assume GPR is the same FAget_by_id('3OAS50')synthase: RTMO12345
model.reactions.EAR130x.gene_reaction_rule = 'RTMO12345'

In [88]:
model.reactions.EAR130x.add_metabolites({
    model.metabolites.ttdec2eACP_c:-1,
    model.metabolites.nadh_c:-1,
    model.metabolites.nad_c:1,
    model.metabolites.tdecACP_c:1,
    model.metabolites.h_c:-1
})

In [89]:
#add reaction with NADH
model.add_reaction(Reaction(id='EAR130y', name = 'Tridecanoyl-[acp]:NADP+ trans-2-oxidoreductase'))

In [90]:
model.reactions.EAR130y.annotation['sbo'] = 'SBO:0000176'

In [91]:
#assume GPR is the same FAsynthase: RTMO12345
model.reactions.EAR130y.gene_reaction_rule = 'RTMO12345'

In [92]:
model.reactions.EAR130y.add_metabolites({
    model.metabolites.ttdec2eACP_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.tdecACP_c:1,
    model.metabolites.h_c:-1
})

In [93]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Finished up to C13:0, now onto C15:0.

In [94]:
#add condenstation of non-ACP with malonyl ACP to 3-oxopentadecnoyl-ACP
#add metabolite
model.add_metabolites(Metabolite(id='3opdecACP_c', name = '3-oxopentadecanoyl-ACP', compartment = 'c', charge =-1, formula = 'C26H47O9N2PRS'))

In [95]:
model.metabolites.get_by_id('3opdecACP_c').annotation['sbo'] = 'SBO:0000176'

In [96]:
#add reaction
model.add_reaction(Reaction(id='3OAS150', name = '3-oxopentadecanoyl-[acyl-carrier-protein] synthase (n-C15:0)'))

In [97]:
model.reactions.get_by_id('3OAS150').annotation['sbo'] = 'SBO:0000176'

In [98]:
#assume GPR is the same FAget_by_id('3OAS50')synthase: RTMO12345
model.reactions.get_by_id('3OAS150').gene_reaction_rule = 'RTMO12345'

In [99]:
model.reactions.get_by_id('3OAS150').add_metabolites({
    model.metabolites.malACP_c:-1,
    model.metabolites.tdecACP_c:-1,
    model.metabolites.ACP_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.get_by_id('3opdecACP_c'):1,
    model.metabolites.h_c:-1
})

In [100]:
#next add reaction with NADPH
#first add 3-Hydroxypentadecanoyl-[acp] met
model.add_metabolites(Metabolite(id='3hpdecACP_c', name = '3-Hydroxypentadecanoyl-[acp]', compartment = 'c', charge = -1, formula = 'C26H49O9N2PRS'))

In [101]:
model.metabolites.get_by_id('3hpdecACP_c').annotation['sbo'] = 'SBO:0000247'

In [102]:
#add reaction
model.add_reaction(Reaction(id='3OAR150', name = ' 3-oxoacyl-[acyl-carrier-protein] reductase (n-C15:0)'))

In [103]:
model.reactions.get_by_id('3OAR150').annotation['sbo'] = 'SBO:0000176'

In [104]:
#assume GPR is the same FA synthase, as done in the whole model: RTMO12345
model.reactions.get_by_id('3OAR150').gene_reaction_rule = 'RTMO12345'

In [105]:
model.reactions.get_by_id('3OAR150').add_metabolites({
    model.metabolites.get_by_id('3opdecACP_c'):-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3hpdecACP_c'):1,
    model.metabolites.h_c:-1
})

In [106]:
#next add the next metabolite
model.add_metabolites(Metabolite(id='tpdec2eACP_c', name = 'Trans-Pentadec-2-enoyl-[acyl-carrier protein]', compartment = 'c', charge = -1, formula = 'C26H47O8N2PRS'))

In [107]:
model.metabolites.get_by_id('tpdec2eACP_c').annotation['sbo'] = 'SBO:0000247'

In [108]:
#add reaction
model.add_reaction(Reaction(id='3HAD150', name = '3-hydroxyacyl-[acyl-carrier-protein] dehydratase (n-C15:0)'))

In [109]:
model.reactions.get_by_id('3HAD150').annotation['sbo'] = 'SBO:0000176'

In [110]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3HAD150').gene_reaction_rule = 'RTMO12345'

In [111]:
model.reactions.get_by_id('3HAD150').add_metabolites({
    model.metabolites.get_by_id('3hpdecACP_c'):-1,
    model.metabolites.h2o_c:1,
    model.metabolites.tpdec2eACP_c:1
})

In [112]:
#add the last reaction both with NADH and NADPH.
#first add the pentadecanoyl-ACP metabolite
model.add_metabolites(Metabolite(id='pdecACP_c', name = 'Pentadecanoyl-ACP(n-C15:0ACP)', compartment = 'c', charge = -1, formula = 'C26H49O8N2PRS' ))

In [113]:
model.metabolites.pdecACP_c.annotation['sbo'] = 'SBO:0000247'

In [114]:
#add reaction with NADH
model.add_reaction(Reaction(id='EAR150x', name = 'Pentadecanoyl-[acp]:NAD+ trans-2-oxidoreductase'))

In [115]:
model.reactions.EAR150x.annotation['sbo'] = 'SBO:0000176'

In [116]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.EAR150x.gene_reaction_rule = 'RTMO12345'

In [117]:
model.reactions.EAR150x.add_metabolites({
    model.metabolites.tpdec2eACP_c:-1,
    model.metabolites.nadh_c:-1,
    model.metabolites.nad_c:1,
    model.metabolites.pdecACP_c:1,
    model.metabolites.h_c:-1
})

In [118]:
#add reaction with NADH
model.add_reaction(Reaction(id='EAR150y', name = 'Pentadecanoyl-[acp]:NADP+ trans-2-oxidoreductase'))

In [119]:
model.reactions.EAR150y.annotation['sbo'] = 'SBO:0000176'

In [120]:
#assume GPR is the same FAsynthase: RTMO12345
model.reactions.EAR150y.gene_reaction_rule = 'RTMO12345'

In [121]:
model.reactions.EAR150y.add_metabolites({
    model.metabolites.tpdec2eACP_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.pdecACP_c:1,
    model.metabolites.h_c:-1
})

In [122]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Finished to nC15:0, now onto C17:0. 

In [123]:
#add condenstation of non-ACP with malonyl ACP to 3-oxoheptadecnoyl-ACP
#add metabolite
model.add_metabolites(Metabolite(id='3ohdecACP_c', name = '3-oxoheptadecnoyl-ACP', compartment = 'c', charge =-1, formula = 'C28H51O9N2PRS'))

In [124]:
model.metabolites.get_by_id('3ohdecACP_c').annotation['sbo'] = 'SBO:0000176'

In [125]:
#add reaction
model.add_reaction(Reaction(id='3OAS170', name = '3-oxoheptadecanoyl-[acyl-carrier-protein] synthase (n-C17:0)'))

In [126]:
model.reactions.get_by_id('3OAS170').annotation['sbo'] = 'SBO:0000176'

In [127]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3OAS170').gene_reaction_rule = 'RTMO12345'

In [128]:
model.reactions.get_by_id('3OAS170').add_metabolites({
    model.metabolites.malACP_c:-1,
    model.metabolites.pdecACP_c:-1,
    model.metabolites.ACP_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.get_by_id('3ohdecACP_c'):1,
    model.metabolites.h_c:-1
})

In [129]:
#next add reaction with NADPH
#first add 3-Hydroxyheptadecanoyl-[acp] met
model.add_metabolites(Metabolite(id='3hhdecACP_c', name = '3-Hydroxyheptadecanoyl-[acp]', compartment = 'c', charge = -1, formula = 'C28H53O9N2PRS'))

In [130]:
model.metabolites.get_by_id('3hhdecACP_c').annotation['sbo'] = 'SBO:0000247'

In [131]:
#add reaction
model.add_reaction(Reaction(id='3OAR170', name = ' 3-oxoacyl-[acyl-carrier-protein] reductase (n-C17:0)'))

In [132]:
model.reactions.get_by_id('3OAR170').annotation['sbo'] = 'SBO:0000176'

In [133]:
#assume GPR is the same FA synthase, as done in the whole model: RTMO12345
model.reactions.get_by_id('3OAR170').gene_reaction_rule = 'RTMO12345'

In [134]:
model.reactions.get_by_id('3OAR170').add_metabolites({
    model.metabolites.get_by_id('3ohdecACP_c'):-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3hhdecACP_c'):1,
    model.metabolites.h_c:-1
})

In [135]:
#next add the next metabolite
model.add_metabolites(Metabolite(id='thdec2eACP_c', name = 'Trans-Heptadec-2-enoyl-[acyl-carrier protein]', compartment = 'c', charge = -1, formula = 'C28H51O8N2PRS'))

In [136]:
model.metabolites.get_by_id('thdec2eACP_c').annotation['sbo'] = 'SBO:0000247'

In [137]:
#add reaction
model.add_reaction(Reaction(id='3HAD170', name = '3-hydroxyacyl-[acyl-carrier-protein] dehydratase (n-C17:0)'))

In [138]:
model.reactions.get_by_id('3HAD170').annotation['sbo'] = 'SBO:0000176'

In [139]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3HAD170').gene_reaction_rule = 'RTMO12345'

In [140]:
model.reactions.get_by_id('3HAD170').add_metabolites({
    model.metabolites.get_by_id('3hhdecACP_c'):-1,
    model.metabolites.h2o_c:1,
    model.metabolites.thdec2eACP_c:1
})

In [141]:
#add the last reaction both with NADH and NADPH.
#first add the heptadecanoyl-ACP metabolite
model.add_metabolites(Metabolite(id='hdecACP_c', name = 'Heptadecanoyl-ACP(n-C17:0ACP)', compartment = 'c', charge = -1, formula = 'C28H53O8N2PRS' ))

In [142]:
model.metabolites.hdecACP_c.annotation['sbo'] = 'SBO:0000247'

In [143]:
#add reaction with NADH
model.add_reaction(Reaction(id='EAR170x', name = 'Heptadecanoyl-[acp]:NAD+ trans-2-oxidoreductase'))

In [144]:
model.reactions.EAR170x.annotation['sbo'] = 'SBO:0000176'

In [145]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.EAR170x.gene_reaction_rule = 'RTMO12345'

In [146]:
model.reactions.EAR170x.add_metabolites({
    model.metabolites.thdec2eACP_c:-1,
    model.metabolites.nadh_c:-1,
    model.metabolites.nad_c:1,
    model.metabolites.hdecACP_c:1,
    model.metabolites.h_c:-1
})

In [147]:
#add reaction with NADH
model.add_reaction(Reaction(id='EAR170y', name = 'Heptadecanoyl-[acp]:NADP+ trans-2-oxidoreductase'))

In [148]:
model.reactions.EAR170y.annotation['sbo'] = 'SBO:0000176'

In [149]:
#assume GPR is the same FAsynthase: RTMO12345
model.reactions.EAR170y.gene_reaction_rule = 'RTMO12345'

In [150]:
model.reactions.EAR170y.add_metabolites({
    model.metabolites.thdec2eACP_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.hdecACP_c:1,
    model.metabolites.h_c:-1
})

In [151]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Now i finished the saturated, unbranched odd chain fatty acids!

## Branched chain, saturated fatty acids
We also have branched chain, saturated fatty acids in the model. Branched chain fatty acids are made by the same fatty acid synthase complex, but have different primer molecules. Even chain Iso- FAs come from a leucine, where odd chain length iso-fatty acids come from a valine molecule.
Anteiso saturated fatty acids come from isoleucine. 

Here I will have to go simiarly to the way we did for the odd-numbered fatty acids through the whole production chain.

### Ante-iso odd numbered carbon FAs
The FAs we need to make here are ante-C15:0 and ante-C17:0. These consists of a C14 and C16 backbone with a single branch at the anteiso position. They are made from Isoleucine, through 2-methylbutyryl-Coa. This then enters the fatty acid synthase cycle all the way until the final products.

Note: ante-C15:0 is also known as 12-methyltetradecanoic acid (CHEBI:39251).

Our model already has the (S)-2-Methylbutanoyl-CoA metabolite (s2mbcoa_c) that enters the elongation by the fatty acid synthase. So I will just check it is properly synthesized from isoleucine. I see that the 3mop_c metabolite can be made from isoleucine, but we cannot convert the 3mop_c into the s2mbcoa_c metabolite. To do so, I will add the OIVD3 reaction.

In [152]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [5]:
#add reaction to make primer s2bmcoa_c
model.add_reaction(Reaction(id='OIVD3', name = '2-oxoisovalerate dehydrogenase (acylating; 3-methyl-2-oxopentanoate)'))

In [6]:
model.reactions.OIVD3.annotation = model_b_sub.reactions.OIVD3.annotation

In [7]:
model.reactions.OIVD3.add_metabolites({
    model.metabolites.get_by_id('3mop_c'):-1,
    model.metabolites.coa_c:-1,
    model.metabolites.nad_c:-1,
    model.metabolites.s2mbcoa_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.nadh_c:1,
    model.metabolites.h_c:-4
})

Next we need to convert the s2mbcoa_c into s2mbACP_c to start the reactions here.

In [8]:
#add metabolite
model.add_metabolites(Metabolite(id='s2mbACP_c', name = '2-methylbutanoyl-ACP', charge = -1, compartment = 'c', formula = 'C16H29N2O8PRS'))

In [9]:
model.metabolites.s2mbACP_c.annotation['sbo'] = 'SBO:0000176'
model.metabolites.s2mbACP_c.annotation['kegg.compound'] = 'C21129'

In [10]:
#add ACP transfer reaction
model.add_reaction(Reaction(id='2MBATA', name = 'CoA:[acyl-carrier-protein] S-acetyltransferase'))

In [11]:
model.reactions.get_by_id('2MBATA').annotation['sbo'] = 'SBO:0000176'

In [12]:
model.reactions.get_by_id('2MBATA').add_metabolites({
    model.metabolites.s2mbACP_c:1,
    model.metabolites.s2mbcoa_c:-1,
    model.metabolites.ACP_c:-1,
    model.metabolites.coa_c:1,
    model.metabolites.h_c:4
})

Now that we have the primer ready, we can start going through the cycles until it forms the anteiso-C17:0 metabolite. 
first fo from anteiso-C5:0 to anteiso-C7:0.

In [13]:
#add condenstation of s2bmcoa-ACP with malonyl ACP to 3-oxomnoyl-ACP
#add metabolite
model.add_metabolites(Metabolite(id='3o2mhACP_c', name = '3-oxo2methylhexanoyl-ACP', compartment = 'c', charge =-1, formula = 'C18H31N2O9PRS'))

In [14]:
model.metabolites.get_by_id('3o2mhACP_c').annotation['sbo'] = 'SBO:0000176'

In [15]:
#add reaction
model.add_reaction(Reaction(id='3OMAS70', name = '3-oxo2methylhexanoyl-[acyl-carrier-protein] synthase (anteiso-C7:0)'))

In [16]:
model.reactions.get_by_id('3OMAS70').annotation['sbo'] = 'SBO:0000176'

In [17]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3OMAS70').gene_reaction_rule = 'RTMO12345'

In [18]:
model.reactions.get_by_id('3OMAS70').add_metabolites({
    model.metabolites.malACP_c:-1,
    model.metabolites.s2mbACP_c:-1,
    model.metabolites.ACP_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.get_by_id('3o2mhACP_c'):1,
    model.metabolites.h_c:-1
})

In [19]:
#next add reaction with NADPH
#first add 3-Hydroxy-2-methylheptanoyl-[acp] met
model.add_metabolites(Metabolite(id='3h2mhACP_c', name = '3-Hydroxy-2-methylhexanoyl-[acp]', compartment = 'c', charge = -1, formula = 'C18H33N2O9PRS'))

In [20]:
model.metabolites.get_by_id('3h2mhACP_c').annotation['sbo'] = 'SBO:0000247'

In [21]:
#add reaction
model.add_reaction(Reaction(id='3OMAR70', name = ' 3-oxo-methyl-acyl-[acyl-carrier-protein] reductase (anteiso-C7:0)'))

In [22]:
model.reactions.get_by_id('3OMAR70').annotation['sbo'] = 'SBO:0000176'

In [23]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3OMAR70').gene_reaction_rule = 'RTMO12345'

In [24]:
model.reactions.get_by_id('3OMAR70').add_metabolites({
    model.metabolites.get_by_id('3o2mhACP_c'):-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3h2mhACP_c'):1,
    model.metabolites.h_c:-1
})

In [25]:
#next add the t2mh2eACP_c metabolite
model.add_metabolites(Metabolite(id='t2mh2eACP_c', name = 'Trans-2-methylhex-2-enoyl-[acyl-carrier protein]', compartment = 'c', charge = -1, formula = 'C18H31N2O8PRS'))

In [26]:
model.metabolites.get_by_id('t2mh2eACP_c').annotation['sbo'] = 'SBO:0000247'

In [27]:
#add reaction
model.add_reaction(Reaction(id='3HMAD70', name = '3-hydroxy-methylacyl-[acyl-carrier-protein] dehydratase (anteiso-C7:0)'))

In [28]:
model.reactions.get_by_id('3HMAD70').annotation['sbo'] = 'SBO:0000176'

In [29]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3HMAD70').gene_reaction_rule = 'RTMO12345'

In [30]:
model.reactions.get_by_id('3HMAD70').add_metabolites({
    model.metabolites.get_by_id('3h2mhACP_c'):-1,
    model.metabolites.h2o_c:1,
    model.metabolites.t2mh2eACP_c:1
})

In [31]:
#last step: convert to 2-methylheptanoyl-ACP which can start the next cycle again
#add both with NADH and NADPH.
#first add the 2-methylhexanoyl-ACP metabolite
model.add_metabolites(Metabolite(id='2mhexACP_c', name = '2methyl-hexanoyl-ACP(anteiso-C7:0ACP)', compartment = 'c', charge = -1, formula = 'C18H33N2O8PRS' ))

In [32]:
model.metabolites.get_by_id('2mhexACP_c').annotation['sbo'] = 'SBO:0000247'

In [33]:
#add reaction with NADH: EMAR70x
model.add_reaction(Reaction(id='EMAR70x', name = '2-Methyl-Hexanoyl-[acp]:NAD+ trans-2-oxidoreductase'))

In [34]:
model.reactions.EMAR70x.annotation['sbo'] = 'SBO:0000176'

In [35]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.EMAR70x.gene_reaction_rule = 'RTMO12345'

In [36]:
model.reactions.EMAR70x.add_metabolites({
    model.metabolites.t2mh2eACP_c:-1,
    model.metabolites.nadh_c:-1,
    model.metabolites.nad_c:1,
    model.metabolites.get_by_id('2mhexACP_c'):1,
    model.metabolites.h_c:-1
})

In [37]:
#add reaction with NADH: EMAR70y
model.add_reaction(Reaction(id='EMAR70y', name = '2-Methyl-Hexanoyl-[acp]:NADp+ trans-2-oxidoreductase'))

In [38]:
model.reactions.EMAR70y.annotation['sbo'] = 'SBO:0000176'

In [39]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.EMAR70y.gene_reaction_rule = 'RTMO12345'

In [40]:
model.reactions.EMAR70y.add_metabolites({
    model.metabolites.t2mh2eACP_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('2mhexACP_c'):1,
    model.metabolites.h_c:-1
})

In [41]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Finished up to anteiso-C7:0 now. Now move onto the next cycle making anteiso-C9:0. 

In [42]:
#add condenstation of 2mhexACP with malonyl ACP to 3-oxomethyl-octanoyl-ACP
#add metabolite
model.add_metabolites(Metabolite(id='3o2moACP_c', name = '3-oxo2methyloctanoyl-ACP', compartment = 'c', charge =-1, formula = 'C20H35N2O9PRS'))

In [43]:
model.metabolites.get_by_id('3o2moACP_c').annotation['sbo'] = 'SBO:0000176'

In [44]:
#add reaction
model.add_reaction(Reaction(id='3OMAS90', name = '3-oxo2methyloctanoyl-[acyl-carrier-protein] synthase (anteiso-C9:0)'))

In [45]:
model.reactions.get_by_id('3OMAS90').annotation['sbo'] = 'SBO:0000176'

In [46]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3OMAS90').gene_reaction_rule = 'RTMO12345'

In [47]:
model.reactions.get_by_id('3OMAS90').add_metabolites({
    model.metabolites.malACP_c:-1,
    model.metabolites.get_by_id('2mhexACP_c'):-1,
    model.metabolites.ACP_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.get_by_id('3o2moACP_c'):1,
    model.metabolites.h_c:-1
})

In [48]:
#next add reaction with NADPH
#first add 3-Hydroxy-2-methyloctanoyl-[acp] met
model.add_metabolites(Metabolite(id='3h2moACP_c', name = '3-Hydroxy-2-methyloctanoyl-[acp]', compartment = 'c', charge = -1, formula = 'C20H37N2O9PRS'))

In [49]:
model.metabolites.get_by_id('3h2moACP_c').annotation['sbo'] = 'SBO:0000247'

In [50]:
#add reaction
model.add_reaction(Reaction(id='3OMAR90', name = ' 3-oxo-methyl-acyl-[acyl-carrier-protein] reductase (anteiso-C9:0)'))

In [51]:
model.reactions.get_by_id('3OMAR90').annotation['sbo'] = 'SBO:0000176'

In [52]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3OMAR90').gene_reaction_rule = 'RTMO12345'

In [53]:
model.reactions.get_by_id('3OMAR90').add_metabolites({
    model.metabolites.get_by_id('3o2moACP_c'):-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3h2moACP_c'):1,
    model.metabolites.h_c:-1
})

In [54]:
#next add the t2mh2eACP_c metabolite
model.add_metabolites(Metabolite(id='t2mo2eACP_c', name = 'Trans-2-methyloct-2-enoyl-[acyl-carrier protein]', compartment = 'c', charge = -1, formula = 'C20H35N2O8PRS'))

In [55]:
model.metabolites.get_by_id('t2mo2eACP_c').annotation['sbo'] = 'SBO:0000247'

In [56]:
#add reaction
model.add_reaction(Reaction(id='3HMAD90', name = '3-hydroxy-methylacyl-[acyl-carrier-protein] dehydratase (anteiso-C9:0)'))

In [57]:
model.reactions.get_by_id('3HMAD90').annotation['sbo'] = 'SBO:0000176'

In [58]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3HMAD90').gene_reaction_rule = 'RTMO12345'

In [59]:
model.reactions.get_by_id('3HMAD90').add_metabolites({
    model.metabolites.get_by_id('3h2moACP_c'):-1,
    model.metabolites.h2o_c:1,
    model.metabolites.t2mo2eACP_c:1
})

In [60]:
#last step: convert to 2-methyloctanoyl-ACP which can start the next cycle again
#add both with NADH and NADPH.
#first add the 2-methyloctanoyl-ACP metabolite
model.add_metabolites(Metabolite(id='2moctACP_c', name = '2methyl-octanoyl-ACP(anteiso-C9:0ACP)', compartment = 'c', charge = -1, formula = 'C20H37N2O8PRS' ))

In [61]:
model.metabolites.get_by_id('2moctACP_c').annotation['sbo'] = 'SBO:0000247'

In [62]:
#add reaction with NADH: EMAR90x
model.add_reaction(Reaction(id='EMAR90x', name = '2-Methyl-Octanoyl-[acp]:NAD+ trans-2-oxidoreductase'))

In [63]:
model.reactions.EMAR90x.annotation['sbo'] = 'SBO:0000176'

In [64]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.EMAR90x.gene_reaction_rule = 'RTMO12345'

In [65]:
model.reactions.EMAR90x.add_metabolites({
    model.metabolites.t2mo2eACP_c:-1,
    model.metabolites.nadh_c:-1,
    model.metabolites.nad_c:1,
    model.metabolites.get_by_id('2moctACP_c'):1,
    model.metabolites.h_c:-1
})

In [66]:
#add reaction with NADH: EMAR90y
model.add_reaction(Reaction(id='EMAR90y', name = '2-Methyl-Octanoyl-[acp]:NADp+ trans-2-oxidoreductase'))

In [67]:
model.reactions.EMAR90y.annotation['sbo'] = 'SBO:0000176'

In [68]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.EMAR90y.gene_reaction_rule = 'RTMO12345'

In [69]:
model.reactions.EMAR90y.add_metabolites({
    model.metabolites.t2mo2eACP_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('2moctACP_c'):1,
    model.metabolites.h_c:-1
})

In [70]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Finished up at anteiso-C9:0. Now onto anteiso-C11:0. 

In [73]:
#add condenstation of 2moctACP with malonyl ACP to 3-oxomethyl-decanoyl-ACP
#add metabolite
model.add_metabolites(Metabolite(id='3o2mdACP_c', name = '3-oxo2methyldecanoyl-ACP', compartment = 'c', charge =-1, formula = 'C22H39N2O9PRS'))

In [74]:
model.metabolites.get_by_id('3o2mdACP_c').annotation['sbo'] = 'SBO:0000176'

In [75]:
#add reaction
model.add_reaction(Reaction(id='3OMAS110', name = '3-oxo2methyldecanoyl-[acyl-carrier-protein] synthase (anteiso-C11:0)'))

In [76]:
model.reactions.get_by_id('3OMAS110').annotation['sbo'] = 'SBO:0000176'

In [77]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3OMAS110').gene_reaction_rule = 'RTMO12345'

In [78]:
model.reactions.get_by_id('3OMAS110').add_metabolites({
    model.metabolites.malACP_c:-1,
    model.metabolites.get_by_id('2moctACP_c'):-1,
    model.metabolites.ACP_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.get_by_id('3o2mdACP_c'):1,
    model.metabolites.h_c:-1
})

In [79]:
#next add reaction with NADPH
#first add 3-Hydroxy-2-methyldecanoyl-[acp] met
model.add_metabolites(Metabolite(id='3h2mdACP_c', name = '3-Hydroxy-2-methyldecanoyl-[acp]', compartment = 'c', charge = -1, formula = 'C22H41N2O9PRS'))

In [80]:
model.metabolites.get_by_id('3h2mdACP_c').annotation['sbo'] = 'SBO:0000247'

In [81]:
#add reaction
model.add_reaction(Reaction(id='3OMAR110', name = ' 3-oxo-methyl-acyl-[acyl-carrier-protein] reductase (anteiso-C11:0)'))

In [82]:
model.reactions.get_by_id('3OMAR110').annotation['sbo'] = 'SBO:0000176'

In [83]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3OMAR110').gene_reaction_rule = 'RTMO12345'

In [84]:
model.reactions.get_by_id('3OMAR110').add_metabolites({
    model.metabolites.get_by_id('3o2mdACP_c'):-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3h2mdACP_c'):1,
    model.metabolites.h_c:-1
})

In [85]:
#next add the t2md2eACP_c metabolite
model.add_metabolites(Metabolite(id='t2md2eACP_c', name = 'Trans-2-methyldec-2-enoyl-[acyl-carrier protein]', compartment = 'c', charge = -1, formula = 'C22H39N2O8PRS'))

In [86]:
model.metabolites.get_by_id('t2md2eACP_c').annotation['sbo'] = 'SBO:0000247'

In [87]:
#add reaction
model.add_reaction(Reaction(id='3HMAD110', name = '3-hydroxy-methylacyl-[acyl-carrier-protein] dehydratase (anteiso-C11:0)'))

In [88]:
model.reactions.get_by_id('3HMAD110').annotation['sbo'] = 'SBO:0000176'

In [89]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3HMAD110').gene_reaction_rule = 'RTMO12345'

In [90]:
model.reactions.get_by_id('3HMAD110').add_metabolites({
    model.metabolites.get_by_id('3h2mdACP_c'):-1,
    model.metabolites.h2o_c:1,
    model.metabolites.t2md2eACP_c:1
})

In [91]:
#last step: convert to 2-methyldecanoyl-ACP which can start the next cycle again
#add both with NADH and NADPH.
#first add the 2-methyldecanoyl-ACP metabolite
model.add_metabolites(Metabolite(id='2mdecACP_c', name = '2methyl-decanoyl-ACP(anteiso-C11:0ACP)', compartment = 'c', charge = -1, formula = 'C22H41N2O8PRS' ))

In [92]:
model.metabolites.get_by_id('2mdecACP_c').annotation['sbo'] = 'SBO:0000247'

In [93]:
#add reaction with NADH: EMAR90x
model.add_reaction(Reaction(id='EMAR110x', name = '2-Methyl-Decanoyl-[acp]:NAD+ trans-2-oxidoreductase'))

In [94]:
model.reactions.EMAR110x.annotation['sbo'] = 'SBO:0000176'

In [95]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.EMAR110x.gene_reaction_rule = 'RTMO12345'

In [96]:
model.reactions.EMAR110x.add_metabolites({
    model.metabolites.t2md2eACP_c:-1,
    model.metabolites.nadh_c:-1,
    model.metabolites.nad_c:1,
    model.metabolites.get_by_id('2mdecACP_c'):1,
    model.metabolites.h_c:-1
})

In [97]:
model.reactions.EMAR110x.check_mass_balance()

{}

In [98]:
#add reaction with NADH: EMAR110y
model.add_reaction(Reaction(id='EMAR110y', name = '2-Methyl-Decanoyl-[acp]:NADp+ trans-2-oxidoreductase'))

In [99]:
model.reactions.EMAR110y.annotation['sbo'] = 'SBO:0000176'

In [100]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.EMAR110y.gene_reaction_rule = 'RTMO12345'

In [101]:
model.reactions.EMAR110y.add_metabolites({
    model.metabolites.t2md2eACP_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('2mdecACP_c'):1,
    model.metabolites.h_c:-1
})

In [102]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Finished up to anteiso-C11:0. Now onto ante-iso-C13:0.

In [103]:
#add condenstation of 2mdecACP with malonyl ACP to 3-oxomethyl-dodecanoyl-ACP
#add metabolite
model.add_metabolites(Metabolite(id='3o2mddACP_c', name = '3-oxo2methyldodecanoyl-ACP', compartment = 'c', charge =-1, formula = 'C24H43N2O9PRS'))

In [104]:
model.metabolites.get_by_id('3o2mddACP_c').annotation['sbo'] = 'SBO:0000176'

In [105]:
#add reaction
model.add_reaction(Reaction(id='3OMAS130', name = '3-oxo2methyldodecanoyl-[acyl-carrier-protein] synthase (anteiso-C13:0)'))

In [106]:
model.reactions.get_by_id('3OMAS130').annotation['sbo'] = 'SBO:0000176'

In [107]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3OMAS130').gene_reaction_rule = 'RTMO12345'

In [108]:
model.reactions.get_by_id('3OMAS130').add_metabolites({
    model.metabolites.malACP_c:-1,
    model.metabolites.get_by_id('2mdecACP_c'):-1,
    model.metabolites.ACP_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.get_by_id('3o2mddACP_c'):1,
    model.metabolites.h_c:-1
})

In [109]:
#next add reaction with NADPH
#first add 3-Hydroxy-2-methyldodecanoyl-[acp] met
model.add_metabolites(Metabolite(id='3h2mddACP_c', name = '3-Hydroxy-2-methyldodecanoyl-[acp]', compartment = 'c', charge = -1, formula = 'C24H45N2O9PRS'))

In [110]:
model.metabolites.get_by_id('3h2mddACP_c').annotation['sbo'] = 'SBO:0000247'

In [111]:
#add reaction
model.add_reaction(Reaction(id='3OMAR130', name = ' 3-oxo-methyl-acyl-[acyl-carrier-protein] reductase (anteiso-C13:0)'))

In [112]:
model.reactions.get_by_id('3OMAR130').annotation['sbo'] = 'SBO:0000176'

In [113]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3OMAR130').gene_reaction_rule = 'RTMO12345'

In [114]:
model.reactions.get_by_id('3OMAR130').add_metabolites({
    model.metabolites.get_by_id('3o2mddACP_c'):-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3h2mddACP_c'):1,
    model.metabolites.h_c:-1
})

In [115]:
#next add the t2md2eACP_c metabolite
model.add_metabolites(Metabolite(id='t2mdd2eACP_c', name = 'Trans-2-methyldodec-2-enoyl-[acyl-carrier protein]', compartment = 'c', charge = -1, formula = 'C24H43N2O8PRS'))

In [116]:
model.metabolites.get_by_id('t2mdd2eACP_c').annotation['sbo'] = 'SBO:0000247'

In [122]:
#add reaction
model.add_reaction(Reaction(id='3HMAD130', name = '3-hydroxy-methylacyl-[acyl-carrier-protein] dehydratase (anteiso-C13:0)'))

Ignoring reaction '3HMAD130' since it already exists.


In [118]:
model.reactions.get_by_id('3HMAD130').annotation['sbo'] = 'SBO:0000176'

In [119]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3HMAD130').gene_reaction_rule = 'RTMO12345'

In [120]:
model.reactions.get_by_id('3HMAD130').add_metabolites({
    model.metabolites.get_by_id('3h2mddACP_c'):-1,
    model.metabolites.h2o_c:1,
    model.metabolites.t2mdd2eACP_c:1
})

In [121]:
#last step: convert to 2-methyldodecanoyl-ACP which can start the next cycle again
#add both with NADH and NADPH.
#first add the 2-methyldodecanoyl-ACP metabolite
model.add_metabolites(Metabolite(id='2mddecACP_c', name = '2methyl-dodecanoyl-ACP(anteiso-C13:0ACP)', compartment = 'c', charge = -1, formula = 'C24H45N2O8PRS' ))

In [123]:
model.metabolites.get_by_id('2mddecACP_c').annotation['sbo'] = 'SBO:0000247'

In [124]:
#add reaction with NADH: EMAR1300x
model.add_reaction(Reaction(id='EMAR130x', name = '2-Methyl-Dodecanoyl-[acp]:NAD+ trans-2-oxidoreductase'))

In [125]:
model.reactions.EMAR130x.annotation['sbo'] = 'SBO:0000176'

In [126]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.EMAR130x.gene_reaction_rule = 'RTMO12345'

In [127]:
model.reactions.EMAR130x.add_metabolites({
    model.metabolites.t2mdd2eACP_c:-1,
    model.metabolites.nadh_c:-1,
    model.metabolites.nad_c:1,
    model.metabolites.get_by_id('2mddecACP_c'):1,
    model.metabolites.h_c:-1
})

In [128]:
#add reaction with NADH: EMAR130y
model.add_reaction(Reaction(id='EMAR130y', name = '2-Methyl-Dodecanoyl-[acp]:NADp+ trans-2-oxidoreductase'))

In [129]:
model.reactions.EMAR130y.annotation['sbo'] = 'SBO:0000176'

In [130]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.EMAR130y.gene_reaction_rule = 'RTMO12345'

In [131]:
model.reactions.EMAR130y.add_metabolites({
    model.metabolites.t2mdd2eACP_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('2mddecACP_c'):1,
    model.metabolites.h_c:-1
})

In [132]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Now finished until anteiso-C13:0. Moving onto anteiso-C15:0.

In [133]:
#add condenstation of 2mddecACP with malonyl ACP to 3-oxomethyl-tetradecanoyl-ACP
#add metabolite
model.add_metabolites(Metabolite(id='3o2mtdACP_c', name = '3-oxo2methyltetradecanoyl-ACP', compartment = 'c', charge =-1, formula = 'C26H47N2O9PRS'))

In [134]:
model.metabolites.get_by_id('3o2mtdACP_c').annotation['sbo'] = 'SBO:0000176'

In [135]:
#add reaction
model.add_reaction(Reaction(id='3OMAS150', name = '3-oxo2methyltetradecanoyl-[acyl-carrier-protein] synthase (anteiso-C15:0)'))

In [136]:
model.reactions.get_by_id('3OMAS150').annotation['sbo'] = 'SBO:0000176'

In [137]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3OMAS150').gene_reaction_rule = 'RTMO12345'

In [138]:
model.reactions.get_by_id('3OMAS150').add_metabolites({
    model.metabolites.malACP_c:-1,
    model.metabolites.get_by_id('2mddecACP_c'):-1,
    model.metabolites.ACP_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.get_by_id('3o2mtdACP_c'):1,
    model.metabolites.h_c:-1
})

In [139]:
#next add reaction with NADPH
#first add 3-Hydroxy-2-methyltetradecanoyl-[acp] met
model.add_metabolites(Metabolite(id='3h2mtdACP_c', name = '3-Hydroxy-2-methyltetradecanoyl-[acp]', compartment = 'c', charge = -1, formula = 'C26H49N2O9PRS'))

In [140]:
model.metabolites.get_by_id('3h2mtdACP_c').annotation['sbo'] = 'SBO:0000247'

In [141]:
#add reaction
model.add_reaction(Reaction(id='3OMAR150', name = ' 3-oxo-methyl-acyl-[acyl-carrier-protein] reductase (anteiso-C15:0)'))

In [142]:
model.reactions.get_by_id('3OMAR150').annotation['sbo'] = 'SBO:0000176'

In [143]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3OMAR150').gene_reaction_rule = 'RTMO12345'

In [144]:
model.reactions.get_by_id('3OMAR150').add_metabolites({
    model.metabolites.get_by_id('3o2mtdACP_c'):-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3h2mtdACP_c'):1,
    model.metabolites.h_c:-1
})

In [145]:
#next add the t2mtd2eACP_c metabolite
model.add_metabolites(Metabolite(id='t2mtd2eACP_c', name = 'Trans-2-methyltetradec-2-enoyl-[acyl-carrier protein]', compartment = 'c', charge = -1, formula = 'C26H47N2O8PRS'))

In [146]:
model.metabolites.get_by_id('t2mtd2eACP_c').annotation['sbo'] = 'SBO:0000247'

In [147]:
#add reaction
model.add_reaction(Reaction(id='3HMAD150', name = '3-hydroxy-methylacyl-[acyl-carrier-protein] dehydratase (anteiso-C15:0)'))

In [148]:
model.reactions.get_by_id('3HMAD150').annotation['sbo'] = 'SBO:0000176'

In [149]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3HMAD150').gene_reaction_rule = 'RTMO12345'

In [150]:
model.reactions.get_by_id('3HMAD150').add_metabolites({
    model.metabolites.get_by_id('3h2mtdACP_c'):-1,
    model.metabolites.h2o_c:1,
    model.metabolites.t2mtd2eACP_c:1
})

In [151]:
#last step: convert to 2-methyltetradecanoyl-ACP which can start the next cycle again
#add both with NADH and NADPH.
#first add the 2-methyltetradecanoyl-ACP metabolite
model.add_metabolites(Metabolite(id='2mtdecACP_c', name = '2methyl-tetradecanoyl-ACP(anteiso-C15:0ACP)', compartment = 'c', charge = -1, formula = 'C26H49N2O8PRS' ))

In [152]:
model.metabolites.get_by_id('2mtdecACP_c').annotation['sbo'] = 'SBO:0000247'
model.metabolites.get_by_id('2mtdecACP_c').annotation['chebi'] ='CHEBI:39251'

In [153]:
#add reaction with NADH: EMAR1300x
model.add_reaction(Reaction(id='EMAR150x', name = '2-Methyl-Tetradecanoyl-[acp]:NAD+ trans-2-oxidoreductase'))

In [154]:
model.reactions.EMAR150x.annotation['sbo'] = 'SBO:0000176'

In [155]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.EMAR150x.gene_reaction_rule = 'RTMO12345'

In [156]:
model.reactions.EMAR150x.add_metabolites({
    model.metabolites.t2mtd2eACP_c:-1,
    model.metabolites.nadh_c:-1,
    model.metabolites.nad_c:1,
    model.metabolites.get_by_id('2mtdecACP_c'):1,
    model.metabolites.h_c:-1
})

In [157]:
#add reaction with NADH: EMAR150y
model.add_reaction(Reaction(id='EMAR150y', name = '2-Methyl-Tetradecanoyl-[acp]:NADp+ trans-2-oxidoreductase'))

In [158]:
model.reactions.EMAR150y.annotation['sbo'] = 'SBO:0000176'

In [159]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.EMAR150y.gene_reaction_rule = 'RTMO12345'

In [160]:
model.reactions.EMAR150y.add_metabolites({
    model.metabolites.t2mtd2eACP_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('2mtdecACP_c'):1,
    model.metabolites.h_c:-1
})

In [161]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Finished until anteiso-C15:0. Now onto the last anteiso-C17:0. 

In [162]:
#add condenstation of 2mtdecACP with malonyl ACP to 3-oxomethyl-hexadecanoyl-ACP
#add metabolite
model.add_metabolites(Metabolite(id='3o2mhdACP_c', name = '3-oxo2methylhexadecanoyl-ACP', compartment = 'c', charge =-1, formula = 'C28H51N2O9PRS'))

In [163]:
model.metabolites.get_by_id('3o2mhdACP_c').annotation['sbo'] = 'SBO:0000176'

In [164]:
#add reaction
model.add_reaction(Reaction(id='3OMAS170', name = '3-oxo2methylhexadecanoyl-[acyl-carrier-protein] synthase (anteiso-C17:0)'))

In [165]:
model.reactions.get_by_id('3OMAS170').annotation['sbo'] = 'SBO:0000176'

In [166]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3OMAS170').gene_reaction_rule = 'RTMO12345'

In [167]:
model.reactions.get_by_id('3OMAS170').add_metabolites({
    model.metabolites.malACP_c:-1,
    model.metabolites.get_by_id('2mtdecACP_c'):-1,
    model.metabolites.ACP_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.get_by_id('3o2mhdACP_c'):1,
    model.metabolites.h_c:-1
})

In [168]:
#next add reaction with NADPH
#first add 3-Hydroxy-2-methylhexadecanoyl-[acp] met
model.add_metabolites(Metabolite(id='3h2mhdACP_c', name = '3-Hydroxy-2-methylhexadecanoyl-[acp]', compartment = 'c', charge = -1, formula = 'C28H53N2O9PRS'))

In [169]:
model.metabolites.get_by_id('3h2mhdACP_c').annotation['sbo'] = 'SBO:0000247'

In [170]:
#add reaction
model.add_reaction(Reaction(id='3OMAR170', name = ' 3-oxo-methyl-acyl-[acyl-carrier-protein] reductase (anteiso-C17:0)'))

In [171]:
model.reactions.get_by_id('3OMAR170').annotation['sbo'] = 'SBO:0000176'

In [172]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3OMAR170').gene_reaction_rule = 'RTMO12345'

In [173]:
model.reactions.get_by_id('3OMAR170').add_metabolites({
    model.metabolites.get_by_id('3o2mhdACP_c'):-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3h2mhdACP_c'):1,
    model.metabolites.h_c:-1
})

In [174]:
#next add the t2mhd2eACP_c metabolite
model.add_metabolites(Metabolite(id='t2mhd2eACP_c', name = 'Trans-2-methylhexadec-2-enoyl-[acyl-carrier protein]', compartment = 'c', charge = -1, formula = 'C28H51N2O8PRS'))

In [175]:
model.metabolites.get_by_id('t2mhd2eACP_c').annotation['sbo'] = 'SBO:0000247'

In [176]:
#add reaction
model.add_reaction(Reaction(id='3HMAD170', name = '3-hydroxy-methylacyl-[acyl-carrier-protein] dehydratase (anteiso-C17:0)'))

In [177]:
model.reactions.get_by_id('3HMAD170').annotation['sbo'] = 'SBO:0000176'

In [178]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.get_by_id('3HMAD170').gene_reaction_rule = 'RTMO12345'

In [179]:
model.reactions.get_by_id('3HMAD170').add_metabolites({
    model.metabolites.get_by_id('3h2mhdACP_c'):-1,
    model.metabolites.h2o_c:1,
    model.metabolites.t2mhd2eACP_c:1
})

In [180]:
#last step: convert to 2-methylhexadecanoyl-ACP 
#add both with NADH and NADPH.
#first add the 2-methylhexadecanoyl-ACP metabolite
model.add_metabolites(Metabolite(id='2mhdecACP_c', name = '2methyl-hexadecanoyl-ACP(anteiso-C17:0ACP)', compartment = 'c', charge = -1, formula = 'C28H53N2O8PRS' ))

In [181]:
model.metabolites.get_by_id('2mhdecACP_c').annotation['sbo'] = 'SBO:0000247'

In [182]:
#add reaction with NADH: EMAR170x
model.add_reaction(Reaction(id='EMAR170x', name = '2-Methyl-Hexadecanoyl-[acp]:NAD+ trans-2-oxidoreductase'))

In [183]:
model.reactions.EMAR170x.annotation['sbo'] = 'SBO:0000176'

In [184]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.EMAR170x.gene_reaction_rule = 'RTMO12345'

In [185]:
model.reactions.EMAR170x.add_metabolites({
    model.metabolites.t2mhd2eACP_c:-1,
    model.metabolites.nadh_c:-1,
    model.metabolites.nad_c:1,
    model.metabolites.get_by_id('2mhdecACP_c'):1,
    model.metabolites.h_c:-1
})

In [186]:
#add reaction with NADH: EMAR170y
model.add_reaction(Reaction(id='EMAR170y', name = '2-Methyl-Hexadecanoyl-[acp]:NADp+ trans-2-oxidoreductase'))

In [187]:
model.reactions.EMAR170y.annotation['sbo'] = 'SBO:0000176'

In [188]:
#assume GPR is the same FA synthase: RTMO12345
model.reactions.EMAR170y.gene_reaction_rule = 'RTMO12345'

In [189]:
model.reactions.EMAR170y.add_metabolites({
    model.metabolites.t2mhd2eACP_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('2mhdecACP_c'):1,
    model.metabolites.h_c:-1
})

In [190]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

In this notebook I ensure the production of:
- 14:0 : met myrsACP_c
- 16:0: met palACP_c
- 18:0: met ocdcaACP_c
- 15:0: met pdecACP_c
- 17:0: met hdecACP_c
- anteiso-C15:0: met 2mtdecACP_c
- anteiso-C17:0: met 2mhdecACP_c

Notebook 45. Lipid metabolism -II will continue with the ones that we have left to do. 

In [4]:
model.optimize()

,fluxes,reduced_costs
IDPh,1.770060,0.000000e+00
CAT,0.019791,0.000000e+00
PDHam1hi,0.000000,5.464379e-17
HYDA,0.000000,-0.000000e+00
MALHYDRO,0.000000,0.000000e+00
...,...,...
3OMAS170,0.000000,-5.700687e-02
3OMAR170,0.000000,-5.700687e-03
3HMAD170,0.000000,0.000000e+00
EMAR170x,0.000000,-1.368165e-02
